In [ ]:
from pyspark.sql.types import LongType

In [ ]:
# Exam 2020/06/27 - Exercise #2 - Spark SQL

In [ ]:
# Input data and output folders
inputPathPurchases = "exam_ex2_data/Purchases.txt"
outputPathPart1 = "outPart1SQL/"
outputPathPart2 = "outPart2SQL/"

In [ ]:
# *****************************************
# Exercise 2 - Part 1
# *****************************************

In [ ]:
# Read the content of Purchases.txt
purchasesDF = spark.read.load(inputPathPurchases,\
                            format="csv",\
                            header=False,\
                            inferSchema=True)\
.withColumnRenamed("_c0", "MID")\
.withColumnRenamed("_c1", "CustomerId")\
.withColumnRenamed("_c2", "Date")\
.withColumnRenamed("_c3", "Price")\
.cache()

In [ ]:
# Associate purchasesDF to a temporary table
purchasesDF.createOrReplaceTempView("purchases")

In [ ]:
# Compute the income for each MID in year 2019
midsIncomesDF = spark.sql("""SELECT MID, SUM(Price) as Income
FROM purchases
WHERE Date>='2019/01/01' and Date<='2019/12/31'
GROUP BY MID""")\
.cache()

In [ ]:
# Associate midsIncomesDF to a temporary table
midsIncomesDF.createOrReplaceTempView("MidsIncomesTable")

In [ ]:
# Select the MIDs associated with the highest annual income in 2019
selectedMidsDF = spark.sql("""SELECT MID
FROM MidsIncomesTable, (SELECT MAX(Income) as MaxIncome
                        FROM MidsIncomesTable) MaxIncomeTable
WHERE Income=MaxIncome""")

In [ ]:
#selectedMidsDF.show()

In [ ]:
# Store the result
selectedMidsDF.write.csv(outputPathPart1,header=False)

In [ ]:
# *****************************************
# Exercise 2 - Part 2
# *****************************************

In [ ]:
# Define a UDF that extract the year part from Date
def yearFunc(date):
    return int(date.split("/")[0])
    
spark.udf.register("yearFunc", yearFunc, LongType())

In [ ]:
# Select the purchases of the last ten years (i.e., from year 2010 to year 2019) 
# Compute the number of purchases for each MID in each year

In [ ]:
midYearNumPurchasesDF = spark.sql("""SELECT MID, 
yearFunc(Date) as Year,
count(*) as NumPurchases
FROM purchases
WHERE Date>='2010/01/01' and Date<='2019/12/31'
GROUP BY MID, yearFunc(Date)""")\
.cache() 

In [ ]:
# Associate midYearNumPurchasesDF to a temporary table
midYearNumPurchasesDF.createOrReplaceTempView("MidYearNumPurchs")

In [ ]:
# We can provide the hint about the usefulness of the brodcast join
midsMaxAnnualPurchasesDF = spark.sql("""SELECT
/*+ BROADCAST(MaxYearlyPurchases) */ 
MID, 
MidYearNumPurchs.Year,
NumPurchases
FROM MidYearNumPurchs, (SELECT Year, Max(NumPurchases) as MaxNumPurchases
                        FROM MidYearNumPurchs
                        GROUP BY Year) MaxYearlyPurchases
WHERE MidYearNumPurchs.NumPurchases=MaxYearlyPurchases.MaxNumPurchases
  AND MidYearNumPurchs.Year=MaxYearlyPurchases.Year""")

In [ ]:
#midsMaxAnnualPurchasesDF.explain()

In [ ]:
# Count for each MID in how many years it is the most purchased model 

In [ ]:
# Associate midsMaxAnnualPurchasesRDD to a temporary table
midsMaxAnnualPurchasesDF.createOrReplaceTempView("midsMaxAnnual")

In [ ]:
selectedMIDsDFPartB = spark.sql("""SELECT MID
FROM midsMaxAnnual
GROUP BY MID
HAVING COUNT(*)>=2""")

In [ ]:
#selectedMIDsDFPartB.show()

In [ ]:
# Store the result
selectedMIDsDFPartB.write.csv(outputPathPart2,header=False)